# Project
## 3rd Notebook : Elasticsearch
##### Students:
* Lilia IZRI      (DS)
* Yacine MOKHTARI (DS)
* Alexandre COMBEAU (DS)

##### Report
[PENSER A METTRE UN LIEN ICI]


In [1]:
from elasticsearch import Elasticsearch
# from collections import Mapping


# database (es)
es = Elasticsearch("http://host.docker.internal:9200") # Same port when running elasticsearch with docker
# index
index = "filtered_tweets"

### Queries

#### 1. Using the Elasticsearch Python Client

##### Simple query to get 2 tweets just to check that it works as well 

In [2]:
query_body = {
    "match_all": {}
}
es.search(index=index, query=query_body, size=2)

/opt/conda/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 547, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'filtered_tweets',
    '_type': '_doc',
    '_id': '3169376024',
    '_score': 1.0,
    '_source': {'user': 'dinkinflickaa_',
     'tweet': 'RT @TetrisphereJon6: Amber Heard just mentioned Kate Moss on the stand &amp; Johnny Depp’s lawyers celebrated. They just caught her in a lie ab…',
     'date': 'Fri May 06 14:10:41 +0000 2022',
     'lat': '44.933143',
     'lon': '7.540121',
     'hashtags': '',
     'sentiment': '1',
     'id': '3169376024'}},
   {'_index': 'filtered_tweets',
    '_type': '_doc',
    '_id': '590228693',
    '_score': 1.0,
    '_source': {'user': 'NadinetTr',
     'tweet': '@lawyerschiff that is false https://t.co/K4D50BO2kB https://t.co/EafsRBuKau',
     'date': 'Fri May 06 14:10:42 +0000 2022',
     'lat': '61.0666922',
     'lon': '-107.991707',
     'hashtags': '',

##### Query where we check for a term with a boost

In [3]:
term2 = "lawyer"
query_with_boost = {
    "span_multi": {
      "match": {
        "prefix": { "tweet": { "value": term2, "boost": 1.08 } }
      }
    } 
}
es.search(index=index, query=query_with_boost, size=2)

{'took': 5,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 11, 'relation': 'eq'},
  'max_score': 20.22163,
  'hits': [{'_index': 'filtered_tweets',
    '_type': '_doc',
    '_id': '590228693',
    '_score': 20.22163,
    '_source': {'user': 'NadinetTr',
     'tweet': '@lawyerschiff that is false https://t.co/K4D50BO2kB https://t.co/EafsRBuKau',
     'date': 'Fri May 06 14:10:42 +0000 2022',
     'lat': '61.0666922',
     'lon': '-107.991707',
     'hashtags': '',
     'sentiment': '-1',
     'id': '590228693'}},
   {'_index': 'filtered_tweets',
    '_type': '_doc',
    '_id': '1270491551043596289',
    '_score': 16.796377,
    '_source': {'user': 'luminu2020',
     'tweet': 'RT @MissAkuaAfriyie: Johnny Depp’s lawyers just need to read tweets. Fact checkers everywhere and they come with receipts',
     'date': 'Fri May 06 13:29:20 +0000 2022',
     'lat': '44.933143',
     'lon': '7.540121',
     'hashtags': '',
  

##### Check for a term in the text of the tweet or the user 

In [4]:
term3 = "Kate"
query_body = {
    "multi_match": {
        "query": term3,
        "type": "most_fields",
        "fields": ["tweet", "user"]
    }    
}

es.search(index=index, query=query_body, size=2)

{'took': 4,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 17, 'relation': 'eq'},
  'max_score': 3.6380336,
  'hits': [{'_index': 'filtered_tweets',
    '_type': '_doc',
    '_id': '1516518637292511232',
    '_score': 3.6380336,
    '_source': {'user': 'moviezclubs2',
     'tweet': 'What Did Amber Heard Assistant Kate James Say About Johnny Depp? Explained\xa0– https://t.co/tMlrvUh3wO',
     'date': 'Fri May 06 14:11:56 +0000 2022',
     'lat': '44.933143',
     'lon': '7.540121',
     'hashtags': '',
     'sentiment': '0',
     'id': '1516518637292511232'}},
   {'_index': 'filtered_tweets',
    '_type': '_doc',
    '_id': '3059665419',
    '_score': 3.3363068,
    '_source': {'user': 'iffycanfly',
     'tweet': "RT @IceQueenCherie: @stfnysnddn Yes, because Kate Moss is Depp's ex gf.  Amber's testimony made relevant previous relationships, hence JD's…",
     'date': 'Fri May 06 13:28:56 +0000 2022',
     'lat': '3

##### Query with a regexp and highlight

In [5]:
field = "user"
regexp = "no.*"

body_query = {
  "query": {
    "regexp": {
      field: {
        "value": regexp,
        "flags": "ALL",
        "case_insensitive": False,
        "max_determinized_states": 10000,
        "rewrite": "constant_score"
      }
    }
  },
  "highlight" : {    
        "pre_tags" : ["<em>"],
        "post_tags" : ["</em>"],
        "fields" : {
            "user" : {}
        }
   }
}
es.search(index=index, body=body_query, size=2)


/tmp/ipykernel_21971/3593569734.py:24: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.search(index=index, body=body_query, size=2)


{'took': 8,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 3, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'filtered_tweets',
    '_type': '_doc',
    '_id': '197204625',
    '_score': 1.0,
    '_source': {'user': 'NoBombardeenUIO',
     'tweet': 'Johnny Depp:  –"Amber era tan mala que me hizo acompañarle a presentar una demanda ante la FIFA para que a Ecuador… https://t.co/TBIFW32qgc',
     'date': 'Fri May 06 14:11:29 +0000 2022',
     'lat': '25.029422',
     'lon': '-77.36195598496681',
     'hashtags': '',
     'sentiment': '0',
     'id': '197204625'},
    'highlight': {'user': ['<em>NoBombardeenUIO</em>']}},
   {'_index': 'filtered_tweets',
    '_type': '_doc',
    '_id': '1309527365509427206',
    '_score': 1.0,
    '_source': {'user': 'nocturnabelle',
     'tweet': 'RT @hleneeh: the way people jump to use ableist language when discussing depp v. heard is something else',
     'date': 'Fri 

#### 2. Using the extended elasticsearch_dsl API (especially, the search engine in it)

Basically, this is just a "simpler" way to express the previous queries. <br>
We will onnly do one query (search a term in a tweet)

In [6]:
from elasticsearch_dsl import Search

# Search engine
s = Search(using=es, index=index)

##### Query where "heard" appears in the tweet text

In [7]:
term = "heard"
result = s.query('match', tweet=term).execute()
for hit in result:
    print(hit.tweet)

RT @GellertDeppBR: Amber Heard: Na Austrália fui espancada, torturada e est*prada por Johnny Depp.  Amber Heard voltando da Austrália:  #Ju…
RT @samjrakoh: amber heard listening to kate james' testimony of amber's abuse of her/amber heard listening to audio of herself abusing joh…
I would LOVE to see the responses of the Amber Heard sympathisers when they get shown the audio of Heard literally… https://t.co/dZSeoSy6Co
RT @KinelRyan: So strange that Amber Heard is suddenly telling stories that no one has ever heard before about Johnny Depp, that no one can…
RT @SystemicDunking: I feel like I'm watching Amber heard playing the role of Amber heard in a made for tv movie about Johnny Depp. Its wil…
RT @ellisgreg: Believe all women? Even Amber Heard? https://t.co/7OCYTx6qqV
RT @ellisgreg: Believe all women? Even Amber Heard? https://t.co/7OCYTx6qqV
Rayuan Maut Johnny Depp saat Lamar Amber Heard https://t.co/bUmDE5WdTq
Sinestas perdudo enbeo casobde Depp Vs Heard, acá te enteras👇
RT @dougnap